In [1]:
!pip install --upgrade scikit-learn==1.6.1


In [3]:
import sklearn
print(sklearn.__version__)


1.6.1


In [ ]:
# 1. Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import joblib

# 2. Load dataset
df = pd.read_csv("player_data.csv")

# 3. Compute Performance Load
df["performance_load"] = (
    df["games_played"] * df["minutes_played"]
    + df["field_goals_attempted"]
    + df["three_point_field_goals_attempted"]
    + df["free_throws_attempted"]
)

# 4. Rule-based Risk Label Assignment
def assign_risk_label(row):
    injuries = row["total_injuries"]
    injury_type = str(row["most_common_injury"]).lower()
    perf_load = row["performance_load"]

    if injuries <= 2 and injury_type in ["eye", "illness", "none"]:
        return "Low"
    elif 3 <= injuries <= 5 and perf_load < 10000:
        return "Low-Medium"
    elif 3 <= injuries <= 5 and perf_load >= 10000:
        return "Medium"
    elif 6 <= injuries <= 10 and perf_load >= 15000:
        return "Medium-High"
    elif injuries > 10 and perf_load >= 20000:
        return "High"
    else:
        return "Medium"

df["predicted_risk_label"] = df.apply(assign_risk_label, axis=1)

# 5. Encode the target labels
label_encoder = LabelEncoder()
df["risk_encoded"] = label_encoder.fit_transform(df["predicted_risk_label"])

# 6. Features and target
features = [
    "age", "height_in_inches", "weight_in_pounds", "games_played", "minutes_played",
    "field_goals_attempted", "three_point_field_goals_attempted",
    "free_throws_attempted", "steals", "blocks", "fouls"
]
X = df[features]
y = df["risk_encoded"]

# 7. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 8. Models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(probability=True, kernel='rbf', C=1),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# 9. Train and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"\nModel: {name}")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Classification Report:\n", classification_report(y_test, preds, target_names=label_encoder.classes_))

# 10. Save best model (Random Forest in this example)
joblib.dump(models["Random Forest"], "rf_injury_model.joblib")
joblib.dump(label_encoder, "risk_label_encoder.joblib")
print("\n✅ Model and label encoder saved!")
